<a href="https://colab.research.google.com/github/Diwakarmogarala/Capstone/blob/main/Hospital_Price_Prediction/Inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# https://drive.google.com/drive/u/0/folders/11zhBUqnUT6J4IVMLpBcrgz8saXAGJ7Hi

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
df = pd.read_csv("/content/drive/MyDrive/Capstone Project/Hospital_Inpatient_Discharges__SPARCS_2016-2022.csv")

<ipython-input-3-1749632543>:1: DtypeWarning: Columns (0,10,15,17,29,30,31,32,33,34,36,37,38,39) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("/content/drive/MyDrive/Capstone Project/Hospital_Inpatient_Discharges__SPARCS_2016-2022.csv")


In [4]:
pd.set_option('display.max_columns', None)
df.head()

,Health Service Area,Hospital County,Operating Certificate Number,Facility Id,Facility Name,Age Group,Zip Code - 3 digits,Gender,Race,Ethnicity,Length of Stay,Type of Admission,Patient Disposition,Discharge Year,CCS Diagnosis Code,CCS Diagnosis Description,CCS Procedure Code,CCS Procedure Description,APR DRG Code,APR DRG Description,APR MDC Code,APR MDC Description,APR Severity of Illness Code,APR Severity of Illness Description,APR Risk of Mortality,APR Medical Surgical Description,Payment Typology 1,Payment Typology 2,Payment Typology 3,Birth Weight,Abortion Edit Indicator,Emergency Department Indicator,Total Charges,Total Costs,Hospital Service Area,Permanent Facility Id,CCSR Diagnosis Code,CCSR Diagnosis Description,CCSR Procedure Code,CCSR Procedure Description
0,Western NY,Allegany,226700.0,37.0,Cuba Memorial Hospital Inc,70 or Older,147,F,White,Not Span/Hispanic,3,Urgent,Home or Self Care,2016,122.0,Pneumonia (except that caused by tuberculosis ...,0.0,NO PROC,139.0,Other pneumonia,4.0,Diseases and Disorders of the Respiratory System,2.0,Moderate,Moderate,Medical,Medicare,Private Health Insurance,NaN,0,N,N,3913.23,3466.83,NaN,NaN,NaN,NaN,NaN,NaN
1,Western NY,Allegany,226700.0,37.0,Cuba Memorial Hospital Inc,30 to 49,147,M,White,Not Span/Hispanic,2,Elective,Home or Self Care,2016,122.0,Pneumonia (except that caused by tuberculosis ...,0.0,NO PROC,139.0,Other pneumonia,4.0,Diseases and Disorders of the Respiratory System,1.0,Minor,Minor,Medical,Self-Pay,NaN,NaN,0,N,Y,3597.1,3746.8,NaN,NaN,NaN,NaN,NaN,NaN
2,Western NY,Allegany,226700.0,37.0,Cuba Memorial Hospital Inc,50 to 69,147,F,White,Not Span/Hispanic,7,Urgent,Home or Self Care,2016,127.0,Chronic obstructive pulmonary disease and bron...,0.0,NO PROC,140.0,Chronic obstructive pulmonary disease,4.0,Diseases and Disorders of the Respiratory System,1.0,Minor,Minor,Medical,Medicare,Federal/State/Local/VA,NaN,0,N,Y,9149.22,8322.1,NaN,NaN,NaN,NaN,NaN,NaN
3,Western NY,Allegany,226700.0,37.0,Cuba Memorial Hospital Inc,70 or Older,147,M,White,Not Span/Hispanic,4,Urgent,Home or Self Care,2016,122.0,Pneumonia (except that caused by tuberculosis ...,0.0,NO PROC,139.0,Other pneumonia,4.0,Diseases and Disorders of the Respiratory System,2.0,Moderate,Moderate,Medical,Medicare,Medicaid,NaN,0,N,Y,5880.61,5828.87,NaN,NaN,NaN,NaN,NaN,NaN
4,Western NY,Allegany,226700.0,37.0,Cuba Memorial Hospital Inc,50 to 69,147,M,White,Not Span/Hispanic,5,Elective,Home or Self Care,2016,122.0,Pneumonia (except that caused by tuberculosis ...,202.0,ELECTROCARDIOGRAM,139.0,Other pneumonia,4.0,Diseases and Disorders of the Respiratory System,2.0,Moderate,Moderate,Medical,Blue Cross/Blue Shield,NaN,NaN,0,N,Y,7019.76,6604.0,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
df.groupby("Payment Typology 1").count()["Total Charges"].sort_values(ascending=False)

,Total Charges
Payment Typology 1,
Medicare,6134387
Medicaid,4680452
Private Health Insurance,2264602
Blue Cross/Blue Shield,1718663
Self-Pay,281764
"Managed Care, Unspecified",270146
Miscellaneous/Other,177016
Federal/State/Local/VA,119690
Department of Corrections,14446


In [6]:
df["APR MDC Description"].value_counts()

,count
APR MDC Description,
DISEASES AND DISORDERS OF THE CIRCULATORY SYSTEM,1366810
"PREGNANCY, CHILDBIRTH AND THE PUERPERIUM",1147697
NEWBORNS AND OTHER NEONATES WITH CONDITIONS ORIGINATING IN THE PERINATAL PERIOD,1050701
DISEASES AND DISORDERS OF THE RESPIRATORY SYSTEM,1001308
DISEASES AND DISORDERS OF THE MUSCULOSKELETAL SYSTEM AND CONNECTIVE TISSUE,912102
DISEASES AND DISORDERS OF THE DIGESTIVE SYSTEM,844605
INFECTIOUS AND PARASITIC DISEASES (SYSTEMIC OR UNSPECIFIED SITES),788889
DISEASES AND DISORDERS OF THE NERVOUS SYSTEM,733438
Diseases and Disorders of the Circulatory System,593646


In [7]:
insurance_dic = {}
df[df["Payment Typology 1"]=="Medicare"]["APR MDC Description"].value_counts()
for insurance in df["Payment Typology 1"].value_counts().index:
  insurance_dic[insurance]=df[df["Payment Typology 1"]==insurance]["APR MDC Description"].value_counts()
insurance_df = pd.DataFrame(insurance_dic)

insurance_df

,Medicare,Medicaid,Private Health Insurance,Blue Cross/Blue Shield,Self-Pay,"Managed Care, Unspecified",Miscellaneous/Other,Federal/State/Local/VA,Department of Corrections,Unknown
APR MDC Description,,,,,,,,,,
ALCOHOL/DRUG USE AND ALCOHOL/DRUG INDUCED ORGANIC MENTAL DISORDERS,48515,230218,28079,15434,8905,3053,656,1832.0,132.0,NaN
Alcohol/Drug Use and Alcohol/Drug Induced Organic Mental Disorders,19502,102670,13643,7523,6285,643,274,595.0,185.0,724.0
BURNS,1971,4040,1205,898,225,463,817,127.0,26.0,NaN
Burns,722,1594,532,431,174,161,381,25.0,12.0,1.0
DISEASES AND DISORDERS OF THE BLOOD AND BLOOD FORMING ORGANS AND IMMUNOLOGICAL DISORDERS,65222,47265,19420,12456,2334,3762,574,934.0,177.0,NaN
DISEASES AND DISORDERS OF THE CIRCULATORY SYSTEM,865357,223232,129529,99366,17100,15489,4837,10550.0,1350.0,NaN
DISEASES AND DISORDERS OF THE DIGESTIVE SYSTEM,399238,189514,119753,96273,12954,17073,2645,6149.0,1006.0,NaN
"DISEASES AND DISORDERS OF THE EAR, NOSE, MOUTH AND THROAT",50490,43711,19994,13841,2832,3463,1458,896.0,207.0,NaN
DISEASES AND DISORDERS OF THE EYE,6391,5280,2458,1765,522,421,397,120.0,57.0,NaN


In [8]:
plt.figure(figsize=(100,100))
sns.barplot(data=insurance_df)

<Axes: >

In [9]:
df["APR DRG Description"].value_counts()

,count
APR DRG Description,
VAGINAL DELIVERY,651605
"NEONATE BIRTHWT >2499G, NORMAL NEWBORN OR NEONATE W OTHER PROBLEM",547886
"Neonate birthwt >2499g, normal newborn or neonate w other problem",385483
SEPTICEMIA & DISSEMINATED INFECTIONS,362364
"NEONATE BIRTH WEIGHT > 2499 GRAMS, NORMAL NEWBORN OR NEONATE WITH OTHER PROBLEM",347260
...,...
NEONATE WITH ECMO,82
Extensive 3rd degree burns w skin graft,81
"NEONATE, TRANSFERRED < 5 DAYS OLD, NOT BORN HERE",74


In [10]:
drug_dic = {}
df[df["Payment Typology 1"]=="Medicare"]["APR DRG Description"].value_counts()
for insurance in df["Payment Typology 1"].value_counts().index:
  drug_dic[insurance]=df[df["Payment Typology 1"]==insurance]["APR DRG Description"].value_counts()
drug_df = pd.DataFrame(drug_dic)

drug_df

,Medicare,Medicaid,Private Health Insurance,Blue Cross/Blue Shield,Self-Pay,"Managed Care, Unspecified",Miscellaneous/Other,Federal/State/Local/VA,Department of Corrections,Unknown
APR DRG Description,,,,,,,,,,
ABDOMINAL PAIN,6360.0,5661,2483,1921,356.0,569.0,105.0,118.0,30.0,NaN
"ABORTION W D&C, ASPIRATION CURETTAGE OR HYSTEROTOMY",12.0,463,199,90,28.0,40.0,NaN,9.0,NaN,NaN
"ABORTION W/O D&C, ASPIRATION CURETTAGE OR HYSTEROTOMY",24.0,964,350,211,64.0,64.0,5.0,25.0,1.0,NaN
"ABORTION WITH D&C, ASPIRATION CURETTAGE OR HYSTEROTOMY",18.0,1078,391,241,36.0,29.0,NaN,13.0,NaN,NaN
"ABORTION WITHOUT D&C, ASPIRATION CURETTAGE OR HYSTEROTOMY",16.0,940,369,189,27.0,25.0,3.0,19.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
Vaginal delivery w sterilization &/or D&C,64.0,3972,1233,1082,84.0,80.0,7.0,91.0,NaN,1.0
Ventricular shunt procedures,1062.0,611,396,383,27.0,47.0,22.0,14.0,NaN,1.0
Vertigo & other labyrinth disorders,6016.0,2047,1012,849,208.0,93.0,42.0,44.0,11.0,13.0


In [11]:
# !pip install transformers
# !pip install torch

In [12]:
from transformers import AutoTokenizer, AutoModel

model_name = "emilyalsentzer/Bio_ClinicalBERT"  # or replace with your MediBERT variant

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/436M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

In [13]:
text = "Patient shows symptoms of diabetes mellitus and hypertension."

inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)
inputs

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


{'input_ids': tensor([[  101,  5351,  2196,  8006,  1104, 17972,  1143,  6473,  4814,  1105,
           177, 24312, 23826,  1988,   119,   102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [14]:
insurance_df.sort_values(by="Medicaid", ascending=False)

,Medicare,Medicaid,Private Health Insurance,Blue Cross/Blue Shield,Self-Pay,"Managed Care, Unspecified",Miscellaneous/Other,Federal/State/Local/VA,Department of Corrections,Unknown
APR MDC Description,,,,,,,,,,
"PREGNANCY, CHILDBIRTH AND THE PUERPERIUM",8021,544399,301305,227714,11330,39498,2791,12551.0,88.0,NaN
NEWBORNS AND OTHER NEONATES WITH CONDITIONS ORIGINATING IN THE PERINATAL PERIOD,2167,518224,252372,196544,34517,34746,1835,10273.0,23.0,NaN
MENTAL DISEASES AND DISORDERS,119478,250854,73021,30830,11318,9669,1423,4547.0,255.0,NaN
"Pregnancy, Childbirth and the Puerperium",3466,248553,125254,97505,6883,7651,825,5274.0,81.0,143.0
DISEASES AND DISORDERS OF THE RESPIRATORY SYSTEM,544815,234670,99821,75359,13233,17347,7241,8137.0,685.0,NaN
Newborns and Other Neonates with Conditions Originating in the Perinatal Period,421,230897,107445,85080,18769,6339,454,4650.0,30.0,170.0
ALCOHOL/DRUG USE AND ALCOHOL/DRUG INDUCED ORGANIC MENTAL DISORDERS,48515,230218,28079,15434,8905,3053,656,1832.0,132.0,NaN
DISEASES AND DISORDERS OF THE CIRCULATORY SYSTEM,865357,223232,129529,99366,17100,15489,4837,10550.0,1350.0,NaN
DISEASES AND DISORDERS OF THE DIGESTIVE SYSTEM,399238,189514,119753,96273,12954,17073,2645,6149.0,1006.0,NaN
